***
# <font> Data Connectivity and Vault</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> OU OCI Delivery </font></p>

***

## Overview:
Data scientists often need to access information stored from different data sources such as object storage, databases and so on. This notebook demonstrates how to how to create a vault, key and secret along with some CRUD operation. This notebook also demostrates how to connect to an Oracle Autonomous Database (ADB) and pull data from ADB into a notebook. Oracle offers two type of ADB, the Autonomous Data Warehouse (ADW) and the Autonomous Transaction Processing (ATP) databases. In general, there are no differences in how a connection is made to these different types of databases. We will use Autonomous Transaction Processing (ATP) for our use case.

## Dataset:
<p>We will be using "Iris Dataset" from UCI machine learning repo. The link to the dataset is <br> https://archive.ics.uci.edu/ml/datasets/iris 

## Execution Environment:
The General Machine Learning for CPUs conda environment includes Oracle AutoML, libraries for data manipulation, and libraries for supervised machine learning through sklearn, xgboost, and lightGBM. This environment provides a good baseline for generic machine learning tasks. The slug for this environment is "generalml_p37_cpu_v1" <p>

First we will start with importing the libraries, you can definitely read about these libraries and functions in ADS class documentation <br> https://accelerated-data-science.readthedocs.io/en/latest/ads.html#subpackages <br>
https://docs.oracle.com/en-us/iaas/tools/python/2.93.0/api/vault.html

In [1]:
import ads
import logging
import pandas as pd
import warnings
import os
import oci
import base64
import json

from oci.config import from_file

from ads.vault.vault import Vault
from ads.common.auth import default_signer
from urllib.request import urlopen

from oci.key_management import KmsManagementClient
from oci.key_management import KmsManagementClientCompositeOperations
from oci.key_management import KmsVaultClient
from oci.key_management import KmsVaultClientCompositeOperations
from oci.key_management.models import CreateVaultDetails
from oci.key_management.models import KeyShape
from oci.key_management.models import CreateKeyDetails
from oci.vault.models import Base64SecretContentDetails
from oci.vault.models import CreateSecretDetails
from oci.vault import VaultsClientCompositeOperations
from oci.secrets import SecretsClient
from oci.vault import VaultsClient
from oci.vault.models import UpdateSecretDetails
from oci.vault.models import ScheduleSecretDeletionDetails
from oci.key_management.models import ScheduleKeyDeletionDetails
from oci.key_management.models import ScheduleVaultDeletionDetails

from ads.database import connection 
from ads.database.connection import Connector
from ads.dataset.factory import DatasetFactory

warnings.filterwarnings("ignore", category=DeprecationWarning) 
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

Helper Functions <br>
Base64 is used to encode binary data as printable text. This allows you to transport binary over protocols or mediums that cannot handle binary data formats and require simple text.
json.load() takes a file object and returns the json object. It is used to read JSON encoded data from a file and convert it into a Python dictionary and deserialize a file itself i.e. it accepts a file object.

In [2]:
def dict_to_secret(dictionary):
    return base64.b64encode(json.dumps(dictionary).encode('ascii')).decode("ascii")

def secret_to_dict(wallet):
    return json.loads(base64.b64decode(wallet.encode('ascii')).decode('ascii'))

# Access with Vault


The [Oracle Cloud Infrastructure Vault](https://docs.cloud.oracle.com/en-us/iaas/Content/KeyManagement/Concepts/keyoverview.htm) lets you centrally manage the encryption keys that protect your data and the secret credentials that you use to securely access resources.

Vaults securely store master encryption keys and secrets that you might otherwise store in configuration files or in code.

Use the Vault service to exercise control over the lifecycle keys and secrets. Integration with Oracle Cloud Infrastructure Identity and Access Management (IAM) lets you control who and what services can access which keys and secrets and what they can do with those resources. The Oracle Cloud Infrastructure Audit integration gives you a way to monitor key and secret use. Audit tracks administrative actions on vaults, keys, and secrets.

Keys are stored on highly available and durable hardware security modules (HSM) that meet Federal Information Processing Standards (FIPS) 140-2 Security Level 3 security certification. The Vault service uses the Advanced Encryption Standard (AES) as its encryption algorithm and its keys are AES symmetric keys.

### Set Up

A vault, key, and secret need to belong to a compartment. By default, the compartment of this notebook session is used. To set up these resources in a different compartment, enter the compartment's OCID in the compartment_id variable.

**The main use case of Vault for a data scientist is to store a secret, such as an SSH key, database password, or some other credential.** To do this, a vault and key are required. By default, this notebook will create these resources. However, the vault_id and key_id variables can be updated with vault and key OCIDs to use existing resources. 

In [3]:
# Select the configuration file to connect to Oracle Cloud Infrastructure resources
config = from_file(os.path.join(os.path.expanduser("~"), ".oci", "config"), "DEFAULT")

# Select the compartment to create the secrets in. 
# Use the notebook compartment by default
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']

# Enter a vault OCID. Otherwise, one will be created.
# "<vault_id>" and "<key_id>"
vault_id = "<vault_id>"
# Enter a KMS OCID to encrypt the secret. Otherwise, one will be created
key_id = "<key_id>"

For the purposes of this notebook, a secret is stored. The secret is the credentials needed to access a database. The notebook is designed so that any secret can be stored as long as it is in the form of a dictionary. To store your secret, just modify the dictionary.

In [4]:
# # Sample credentials that are going to be stored. This is a sample credential for an Autonomous Database.
# credential = {'database_name': 'DATABASE_high', 
#               'username': 'ADMIN', 
#               'password': 'MacbookAir',
#               'database_type':'oracle'}

### Check Vault under Identity & Security from Navigation Menu for each of the subsequent steps.

### Create a Vault

To store a secret, a key is needed to encrypt and decrypt the secret. This key and secret are stored in a vault. The code in the following cell creates a vault if you have not specified an OCID in the vault_id variable. The KmsVaultClient class takes a configuration object and establishes a connection to the key management service (KMS). Communication with KmsVaultClient is asynchronous. For the purpose of this notebook, it is better to have synchronous communication so the KmsVaultClient are wrapped in a KmsVaultClientCompositeOperations object.

The details of the vault are specified using an object of the CreateVaultDetails type. A compartment ID must be provided along with the properties of the vault. For the purposes of this notebook, the vault's display name is DataScienceVault. The vault name must be unique. This value can be changed to fit your individual needs.

In [5]:
# if vault_id == "<vault_id>":
#     # Create a VaultClientCompositeOperations for composite operations.
#     vault_client = KmsVaultClientCompositeOperations(KmsVaultClient(config))

#     # Create vault_details object for use in creating the vault.
#     vault_details = CreateVaultDetails(compartment_id=compartment_id,
#     vault_type=oci.key_management.models.Vault.VAULT_TYPE_DEFAULT,
#     display_name="DataScienceVaultDemo")

#     # Vault creation is asynchronous; Create the vault and wait until it becomes active.
#     print("Creating vault...", end='')
#     vault = vault_client.create_vault_and_wait_for_state(vault_details,
#                 wait_for_states=[oci.vault.models.Secret.LIFECYCLE_STATE_ACTIVE]).data
#     vault_id = vault.id
#     print('Done')
#     print("Created vault: {}".format(vault_id))
# else:
#     # Get the vault using the vault OCID.
#     vault = KmsVaultClient(config).get_vault(vault_id=vault_id).data
#     print("Using vault: {}".format(vault.id))

### Create a Key

The secret is encrypted and decrypted using an AES key. The code in the following cell creates a key if you have not specified an OCID in the key_id variable. The KmsManagementClient class takes a configuration object and the endpoint for the vault that is going to be used to store the key. It establishes a connection to the KMS. Communication with KmsManagementClient is asynchronous. For the purpose of this notebook, it is better to have synchronous communication so the KmsManagementClient is wrapped in a KmsManagementClientCompositeOperations object.

The details of the key are specified using an object of type CreateKeyDetails. A compartment OCID must be provided along with the properties of the key. The KeyShape class defines the properties of the key. In this example, it is a 32-bit AES key.

For the purposes of this notebook, the key's display name is DataScienceKey. The name of a key must be unique.

In [6]:
# if key_id == "<key_id>":
#     # Create a vault management client using the endpoint in the vault object.
#     vault_management_client = KmsManagementClientCompositeOperations(
#         KmsManagementClient(config, service_endpoint=vault.management_endpoint))

#     # Create key_details object that needs to be passed when creating key.
#     key_details = CreateKeyDetails(compartment_id=compartment_id,
#         display_name="DataScienceKey", 
#         key_shape=KeyShape(algorithm="AES", length=32))

#     # Vault creation is asynchronous; Create the vault and wait until it becomes active.
#     print("Creating key...", end='')
#     key = vault_management_client.create_key_and_wait_for_state(key_details,
#               wait_for_states=[oci.key_management.models.Key.LIFECYCLE_STATE_ENABLED]).data
#     key_id = key.id
#     print('Done')
#     print("Created key: {}".format(key_id))
# else:
#     print("Using key: {}".format(key_id))

### Store a Secret

The code in the following cell creates a secret that is to be stored. The variable credential is a dictionary and contains the information that is to be stored. The UDF dict_to_secret takes a Python dictionary, converts it to a JSON string, and then Base64 encodes it. This string is what is to be stored as a secret so the secret can be parsed by any system that may need it.

The VaultsClient class takes a configuration object and establishes a connection to the Vault service. Communication with VaultsClient is asynchronous. For the purpose of this notebook, it is better to have synchronous communication so VaultsClient is wrapped in a VaultsClientCompositeOperations object.

The contents of the secret are stored in a Base64SecretContentDetails object. This object contains information about the encoding being used, the stage to be used,and most importantly the payload (the secret). The CreateSecretDetails class is used to wrap the Base64SecretContentDetails object and also specify other properties about the secret. It requires the compartment OCID, the vault that is to store the secret, and the key to use to encrypt the secret. For the purposes of this notebook, the secret's display name is DataScienceSecret. The name of a secret must be unique.

In [7]:
# # Encode the secret.
# secret_content_details = Base64SecretContentDetails(
#     content_type=oci.vault.models.SecretContentDetails.CONTENT_TYPE_BASE64,
#     stage=oci.vault.models.SecretContentDetails.STAGE_CURRENT,
#     content=dict_to_secret(credential))
    
# # Bundle the secret and metadata about it.
# secrets_details = CreateSecretDetails(
#         compartment_id=compartment_id,
#         description = "Data Science service test secret", 
#         secret_content=secret_content_details,
#         secret_name="DataScienceSecret",
#         vault_id=vault_id,
#         key_id=key_id)

# # Store secret and wait for the secret to become active.
# print("Creating secret...", end='')
# vaults_client_composite = VaultsClientCompositeOperations(VaultsClient(config))
# secret = vaults_client_composite.create_secret_and_wait_for_state(
#              create_secret_details=secrets_details,
#              wait_for_states=[oci.vault.models.Secret.LIFECYCLE_STATE_ACTIVE]).data
# secret_id = secret.id
# print('Done')
# print("Created secret: {}".format(secret_id))

### Retrieve a Secret

The SecretsClient class takes a configuration object. The get_secret_budle method takes the secret's OCID and returns a Response object. Its data attribute returns SecretBundle object. This has an attribute secret_bundle_content that has the object Base64SecretBundleContentDetails and the content attribute of this object has the actual secret. This returns the Base64 encoded JSON string that was created with the .dict_to_secret() method. The process can be reversed with the .secret_to_dict() method. This will return a dictionary with the secrets.

In [8]:
# secret_bundle = SecretsClient(config).get_secret_bundle(secret_id)
# secret_content = secret_to_dict(secret_bundle.data.secret_bundle_content.content)    

# print(secret_content)

### Update a Secret

Secrets are immutable but it is possible to update them by creating new versions. In the code in the following cell, the credential object updates the password key. To update the secret, a Base64SecretContentDetails object must be created. The process is the same as previously described in the Store a Secret section. However, instead of using a CreateSecretDetails object, an UpdateSecretDetails object is used and only the information that is being changed is passed in.

Note that the OCID of the secret does not change. A new secret version is created and the old secret is rotated out of use, but it may still be available depending on the tenancy configuration.

The code in the following cell updates the secret. It then prints the OCID of the old secret and the new secret (they will be the same). It also retrieves the updated secret, converts it into a dictionary, and prints it. This shows that the password was actually updated.

In [9]:
# # Update the password in the secret.
# credential['password'] = 'MacbookPro'

# # Encode the secret.
# secret_content_details = Base64SecretContentDetails(
#     content_type=oci.vault.models.SecretContentDetails.CONTENT_TYPE_BASE64,
#     stage=oci.vault.models.SecretContentDetails.STAGE_CURRENT,
#     content=dict_to_secret(credential))

# # Store the details to update.
# secrets_details = UpdateSecretDetails(secret_content=secret_content_details)
     
# #Create new secret version and wait for the new version to become active.
# secret_update = vaults_client_composite.update_secret_and_wait_for_state(
#     secret_id, 
#     secrets_details,
#     wait_for_states=[oci.vault.models.Secret.LIFECYCLE_STATE_ACTIVE]).data

# # The secret OCID does not change.
# print("Orginal Secret OCID: {}".format(secret_id))
# print("Updated Secret OCID: {}".format(secret_update.id))

# ### Read a secret's value.
# secret_bundle = SecretsClient(config).get_secret_bundle(secret_update.id)
# secret_content = secret_to_dict(secret_bundle.data.secret_bundle_content.content)    

# print(secret_content)

### Delete a Secret

Vaults, keys, and secrets cannot be deleted immediately. They are marked as pending deletion. By default, they are deleted 30 days after they request for deletion. The length of time before deletion is configurable.

The schedule_secret_deletion method of the VaultsClient class is used to delete a secret. It requires the secret's OCID and a ScheduleSecretDeletionDetails object. The ScheduleSecretDeletionDetails provides details about when the secret is deleted.

The schedule_secret_deletion method returns a Response object that has information about the deletion process. If the key has already been marked for deletion, a ServiceError occurs with information about the key.

In [10]:
# try:
#     VaultsClient(config).schedule_secret_deletion(secret_id, ScheduleSecretDeletionDetails())
# except:
#     print("The secret has already been deleted?")

## Autonomous Database with Vault

Oracle offers two type of ADB, the Autonomous Data Warehouse (ADW) and the Autonomous Transaction Processing (ATP) databases. In general, there are no differences in how a connection is made to these different types of databases.


The [Oracle Cloud Infrastructure Vault](https://docs.cloud.oracle.com/en-us/iaas/Content/KeyManagement/Concepts/keyoverview.htm) lets you centrally manage the encryption keys that protect your data and the secret credentials that you use to securely access resources.

Vaults securely store master encryption keys and secrets that you might otherwise store in configuration files or in code.

Use the Vault service to exercise control over the lifecycle keys and secrets. Integration with Oracle Cloud Infrastructure Identity and Access Management (IAM) lets you control who and what services can access which keys and secrets and what they can do with those resources. The Oracle Cloud Infrastructure Audit integration gives you a way to monitor key and secret use. Audit tracks administrative actions on vaults, keys, and secrets.

Keys are stored on highly available and durable hardware security modules (HSM) that meet Federal Information Processing Standards (FIPS) 140-2 Security Level 3 security certification. The Vault service uses the Advanced Encryption Standard (AES) as its encryption algorithm and its keys are AES symmetric keys.

<a id='setup'></a>
## Setting Up the Notebook Session to Access ADB

To access ADB, (ADW or ATP databases) from the notebook environment, the database's wallet and user credentials will be needed for our demo. You can refer ADS documentation if you want to do it without the wallet. The wallet is a ZIP file that contains connection information and the encryption keys that are needed to establish a secure connection to the database.

The following instructions are for the ADW though the steps are identical for an ATP database. 

Upload the wallet to the notebook. The wallet can be obtained from your database administrator, Oracle Cloud Infrastructure API, Oracle CLI, or from the Console. In this example, it is downloaded from the Console.

<a id='setup_01'></a>
### 1. Go to the database console

In the Console, navigate to the **Autonomous Data Warehouse** or **Autonomous Transaction Processing** section. 

<a id='setup_02'></a>
### 2. Select the database

Select the database to connect to. If one does not exist, one may need to be created. Follow Step 3 to create otherwise skip tp Step 5

<a id='setup_03'></a>
### 3. Create database

On the main page, click on create Autonomous Database:
Next, Provide basic information for the Autonomous Database such as Compartment, Display name and Database name.
Select Transaction Processing for our usecase.
Choose Shared infrastructure as deployment type.
Provide administrator credentials.
Click on Create.

<a id='setup_04'></a>
### 4. Upload data

On the main page of the database, once the database is active, click **Database Actions**.
On the new page, click on SQL in the Development section.
If prompted, provide credentials created in step 3.
you will see an SQL development environment, upload the iris.csv through Data Loading.


<a id='setup_05'></a>
### 5. Open DB Connection

On the main page of the database, click **DB Connection** on an active database.

<a id='setup_06'></a>
### 6. Download the wallet

Click **Download Wallet**. You are asked to create a password.

<a id='setup_07'></a>
### 7. Check the wallet

It has a name like `Wallet_<database_name>.zip`. For example, `Wallet_production.zip`.

<a id='setup_08'></a>
### 8. Upload the wallet to the notebook

To upload the wallet, drag and drop it into the file browser, or click upload.

<a id='setup_09'></a>
### 9. Creating credentials

`ADS` methods for working with credentials uses a Python `dict` to store the key/value pairs. Therefore, any arbitrary values can be stored. Below is a common use case to store credentials to access an Oracle Autonomous Database.

In the `credential` variable, the following values can be updated to store the desired authentication information; `<service_name>`, `<user_name>`, `<password>` and `<wallet_location>`.

Usually for ADW you will have identifiers endings with `_high`, `_medium` and `_low`. All 3 will connect to the same database, only difference is in how they allocate database resources.

In [55]:
# Sample credentials that are going to be stored.
credential1 = {'service_name': 'dbtester_high',
              'user_name': 'ADMIN', 
              'password': 'MacbookPro123',
               'wallet_location':'/home/datascience/Wallet_dbtester_high.zip'
              }

## <a id='setup_10'></a>
### 10. Store Credentials

It is best practice to store credentials outside of the notebook. Therefore, the preferred method is to use the Oracle Cloud Infrastructure Vault. If your tenancy is not configured to use the Vault, it is recommended that it be configured as this is the most secure method.

The `ADS` vault methods assume that the data being stored is in a dictionary. Internally it stores it as an encoded JSON object.

In the following cell, update `<vault_id>` with the OCID for the vault that you wish to connect to. To encrypt/decrypt the data an encryption key is needed. This key is part of the vault. If one does not exist you may have to create it. Update `<key_id>` with the OCID of the encryption key.

In [56]:
vault_id = "ocid1.vault.oc1.phx.bzscckuuaaaxo.abyhqljs6urnutgv6rlmajkisrtmklfig3l66dzjzyebojndxfreimtyrjxa"
key_id = "ocid1.key.oc1.phx.bzscckuuaaaxo.abyhqljtwmvxgxyfwem6sgzqobu4iawox5hlrddbevdcjv6wkcsmlpiii3xa"
# secret_id = "ocid1.vaultsecret.oc1.phx.amaaaaaanf25m3qad5mb43c72bejtvuppqg4bshvlo5hja4i226s4u6tbbuq"
# secret_id = "ocid1.vaultsecret.oc1.phx.amaaaaaanf25m3qa6lltdedajws26l7ldjiz7amdb6jubvzl2luhfgr7dura"
secret_id = "<secret_id>"
print(vault_id)
print(key_id)
print(secret_id)

ocid1.vault.oc1.phx.bzscckuuaaaxo.abyhqljs6urnutgv6rlmajkisrtmklfig3l66dzjzyebojndxfreimtyrjxa
ocid1.key.oc1.phx.bzscckuuaaaxo.abyhqljtwmvxgxyfwem6sgzqobu4iawox5hlrddbevdcjv6wkcsmlpiii3xa
<secret_id>


The following cell obtains a handle to the vault.

In [57]:
if vault_id != "<vault_id>" and key_id != "<key_id>":
    vault = Vault(vault_id=vault_id, key_id=key_id)
    secret_id = vault.create_secret(credential1)

Create a secret if you do not have it. It returns the OCID to the secret so that it can be accessed.

There are a few attributes on the Vault class that provide information about the actual vault connections.

In [58]:
if "vault" in globals():
    print(f"Vault OCID: {vault.id}")
    print(f"Encryption Key OCID: {vault.key_id}")
    print(f"Compartment OCID: {vault.compartment_id}")
    print(f"Secret OCID: {secret_id}")
    print(f"Secret: {vault.get_secret(secret_id)}")
else:
    print("Skipping as it appears that you do not have vault configured.")

Vault OCID: ocid1.vault.oc1.phx.bzscckuuaaaxo.abyhqljs6urnutgv6rlmajkisrtmklfig3l66dzjzyebojndxfreimtyrjxa
Encryption Key OCID: ocid1.key.oc1.phx.bzscckuuaaaxo.abyhqljtwmvxgxyfwem6sgzqobu4iawox5hlrddbevdcjv6wkcsmlpiii3xa
Compartment OCID: ocid1.compartment.oc1..aaaaaaaarxbwpf4b5mquu54ibgl7ne2dukgddm5uzhroou6ofu342i54orza
Secret OCID: ocid1.vaultsecret.oc1.phx.amaaaaaanf25m3qaop46gblsnlbvktdxcrb4gdmppzjdeis3fxmyxc6m5bya
Secret: {'service_name': 'dbtester_high', 'user_name': 'ADMIN', 'password': 'MacbookPro123', 'wallet_location': '/home/datascience/Wallet_dbtester_high.zip'}


In [59]:
database_name = "dbtester"
wallet_path = os.path.join(os.path.expanduser("~"), f"Wallet_{database_name}_high.zip")
print(wallet_path)

/home/datascience/Wallet_dbtester_high.zip


In [60]:
try:
    urlopen('http://oracle.com', timeout=5)
    print("There is public internet access")
except:
    print("There is no public internet access")

There is public internet access


### <a id='setup_11'></a>
### 11. Connect to Autonomous Database

To make a connection using a secret in the Vault, set the parameter secret_id to the secret OCID. The get_secret_budle method takes the secret's OCID and returns a Response object. Its data attribute returns SecretBundle object. This has an attribute secret_bundle_content that has the object Base64SecretBundleContentDetails and the content attribute of this object has the actual secret. 

In [61]:
secret_bundle = SecretsClient(config).get_secret_bundle(secret_id)
secret_content = secret_to_dict(secret_bundle.data.secret_bundle_content.content) 
print(secret_content)

{'service_name': 'dbtester_high', 'user_name': 'ADMIN', 'password': 'MacbookPro123', 'wallet_location': '/home/datascience/Wallet_dbtester_high.zip'}


In [63]:
# simple read of a SQL query into a dataframe with no bind variables
df = pd.DataFrame.ads.read_sql(
    "SELECT * FROM IRIS",
    connection_parameters=secret_content,
)

In [64]:
df.head()

,ID,SEPALLENGTHCM,SEPALWIDTHCM,PETALLENGTHCM,PETALWIDTHCM,SPECIES
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
